In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from typing import Any, Sequence, Optional, Tuple, Iterator, Dict, Callable, Union, Type, List
import numpy as np
import matplotlib.pyplot as plt
import pprint

# Changing fonts to be latex typesetting
from matplotlib import rcParams
rcParams['mathtext.fontset'] = 'dejavuserif'
rcParams['font.family'] = 'serif'

# JAX/Flax
import jax
import jax.numpy as jnp
from jax import random
import wandb

from visualization import visualize as vis
from data import data_functions as df
from models import model_architecture as march
from models import train_model as trm
from models import generate_model as gen

In [10]:
sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'Loss',
        'goal': 'minimize'
    },
    'parameters': {
        'optimizer': {
            'value': 'adam'
        },
        'embedding_dim': {
            'values': [32, 64, 128, 256]
        },
        'epochs': {
            'value': 30
        },
        'batch_size': {
            'values': [64, 128, 256]
        },
        'learning_rate': {
            'values': [1e-3, 1e-4, 1e-5]
        },
        'N': {
            'value': 2
        },
        'D': {
            'values': [100, 1000, 2000, 4000]
        },
        'std_data': {
            'value': 0.5
        },
        'depth': {
            'values': [2, 3, 4, 5, 6]
        },
        'initial_feature': {
            'values': [16, 32, 64, 128, 256]
        },
        'out_feature': {
            'value': 2
        },
        'size': {
            'values': [2000, 4000]
        },
        'gen_data_key': {
            'value': 21
        },
        'train_model_key': {
            'value': 47
        }
    }
}

pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'Loss'},
 'parameters': {'D': {'values': [100, 1000, 2000, 4000]},
                'N': {'value': 2},
                'batch_size': {'values': [64, 128, 256]},
                'depth': {'values': [2, 3, 4, 5, 6]},
                'embedding_dim': {'values': [32, 64, 128, 256]},
                'epochs': {'value': 30},
                'gen_data_key': {'value': 21},
                'initial_feature': {'values': [16, 32, 64, 128, 256]},
                'learning_rate': {'values': [0.001, 0.0001, 1e-05]},
                'optimizer': {'value': 'adam'},
                'out_feature': {'value': 2},
                'size': {'values': [2000, 4000]},
                'std_data': {'value': 0.5},
                'train_model_key': {'value': 47}}}


In [11]:
def train_sweep(config=None):
    # Create random PRNG keys for training
    init_rng = random.PRNGKey(0)
    rng, subkey1, subkey2, subkey3 = random.split(init_rng, num=4)

    # Get the current directory (notebooks directory in your case)
    current_dir = os.getcwd()
    # Move up one level to the parent directory (PFGMPP)
    parent_dir = os.path.dirname(current_dir)
    # Construct the wandb directory path
    wandb_dir = os.path.join(parent_dir, 'wandb')
    
    # Initialize a new wandb run
    with wandb.init(config=config, dir=wandb_dir):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        
        # Generate dataset of 2D Gaussians
        X = df.generate_data(config['gen_data_key'], config['size'])
        
        # Make the data suitable for a JAX Dataloader
        train_dataset = df.JaxDataset(X=X)
        train_loader = df.NumpyLoader(dataset=train_dataset,
                                      batch_size=config['batch_size'],
                                      shuffle=True)
        batch = next(iter(train_loader))
        
        # Get the rng & model instantiated
        model = march.UNet(depth=config['depth'],
                           initial_feature=config['initial_feature'],
                           std_data=config['std_data'],
                           out_feature=config['out_feature'],
                           embedding_dim=config['embedding_dim'])
        
        # Sample the noise distribution
        rnd_normal = random.normal(subkey2, shape=(batch.shape[0], 1))
        t = jnp.exp(rnd_normal * 1.2 - 1.2)  
    
    
        # Initialize the models state
        state = trm.init_train_state(model=model,
                                     random_key=subkey3,
                                     x_shape=batch.shape,
                                     t_shape=t.shape,
                                     learning_rate=config['learning_rate'])
        
        model, state = trm.train_model_sweep(train_loader,
                                             model,
                                             state,
                                             config)

        del state
        del model
        del train_dataset
        del train_loader
        del X
        del t 
        del batch
        wandb.finish()

In [12]:
sweep_id = wandb.sweep(sweep_config, project="toy_pfgmpp")
wandb.agent(sweep_id, train_sweep, count=20)

Create sweep with ID: 2vch61jw
Sweep URL: https://wandb.ai/mdowicz/toy_pfgmpp/sweeps/2vch61jw


wandb: Agent Starting Run: 42ig95xw with config:
wandb: 	D: 100
wandb: 	N: 2
wandb: 	batch_size: 256
wandb: 	depth: 5
wandb: 	embedding_dim: 128
wandb: 	epochs: 30
wandb: 	gen_data_key: 21
wandb: 	initial_feature: 256
wandb: 	learning_rate: 1e-05
wandb: 	optimizer: adam
wandb: 	out_feature: 2
wandb: 	size: 2000
wandb: 	std_data: 0.5
wandb: 	train_model_key: 47
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


100%|██████████| 30/30 [02:19<00:00,  4.66s/it]


Train Loss,███▇▆▆▅▄▃▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train Loss,22.62677


wandb: Agent Starting Run: nx9amuel with config:
wandb: 	D: 4000
wandb: 	N: 2
wandb: 	batch_size: 64
wandb: 	depth: 3
wandb: 	embedding_dim: 32
wandb: 	epochs: 30
wandb: 	gen_data_key: 21
wandb: 	initial_feature: 64
wandb: 	learning_rate: 0.001
wandb: 	optimizer: adam
wandb: 	out_feature: 2
wandb: 	size: 4000
wandb: 	std_data: 0.5
wandb: 	train_model_key: 47
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


...

100%|██████████| 30/30 [02:06<00:00,  4.23s/it]


Train Loss,█▇▅▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train Loss,6.32395


wandb: Agent Starting Run: fgd49wzz with config:
wandb: 	D: 4000
wandb: 	N: 2
wandb: 	batch_size: 256
wandb: 	depth: 6
wandb: 	embedding_dim: 256
wandb: 	epochs: 30
wandb: 	gen_data_key: 21
wandb: 	initial_feature: 256
wandb: 	learning_rate: 0.0001
wandb: 	optimizer: adam
wandb: 	out_feature: 2
wandb: 	size: 4000
wandb: 	std_data: 0.5
wandb: 	train_model_key: 47
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


  0%|          | 0/30 [00:13<?, ?it/s]


Run fgd49wzz errored: XlaRuntimeError('RESOURCE_EXHAUSTED: Out of memory while trying to allocate 268435456 bytes.\nBufferAssignment OOM Debugging.\nBufferAssignment stats:\n             parameter allocation:   10.82GiB\n              constant allocation:       588B\n        maybe_live_out allocation:   10.82GiB\n     preallocated temp allocation:    14.1KiB\n                 total allocation:   21.64GiB\n              total fragmentation:    14.7KiB (0.00%)\nPeak buffers:\n\tBuffer 1:\n\t\tSize: 1.00GiB\n\t\tEntry Parameter Subshape: f32[16384,16384]\n\t\t==========================\n\n\tBuffer 2:\n\t\tSize: 1.00GiB\n\t\tEntry Parameter Subshape: f32[16384,16384]\n\t\t==========================\n\n\tBuffer 3:\n\t\tSize: 1.00GiB\n\t\tEntry Parameter Subshape: f32[16384,16384]\n\t\t==========================\n\n\tBuffer 4:\n\t\tSize: 1.00GiB\n\t\tOperator: op_name="jit(train_step)/jit(main)/add" source_file="/home/mdowicz/PFGMPP/src/models/train_model.py" source_line=117\n\t\tXLA Label: 